Mostly not contain the inpatient data (only 10% overlap)

**features**:

*Age bins*: number of unit stays within this age range

age_mean: the mean admission age for each patient

age_std: the std of the admission age for each patient

age_min: the min age among all admissions

age_max: the max age among all admissions

freq: the frequency of lab tests per year


num_cvd_readmission: number of readmissions that are caused by cardiovascular

Cardiovescular: whether the patient has ever had a cardiovescular diagnosis

readmission: whether this patient has been admitted again

num_stays: total number of visits

stay_length: total length of emergency visits (use mean stay length to impute)

stay_min: the min stay length of all admissions

stay_max: the max stay length of all admissions

stay_std: the std of stay length

stay_mean: the mean stay length

died: whether this patient is dead or not 

Use rf to impute the length of stays

age: min, max

visit_length: mean, min, max, std

In [76]:
import pandas as pd
import os
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [77]:
datatype = "test"

In [78]:
if datatype == "train": load_path = "/home/bhatti/dataset/VCHAMPS/ed_visits_train.csv"
elif datatype == "test": load_path = "/data/public/MLA/VCHAMPS-Test/ed_visits_test.csv"
else: load_path = "/data/public/MLA/VCHAMPS-Quality/ed_visits_qual_Formatfixed.csv"
ed = pd.read_csv(load_path, index_col=0)

In [79]:
if datatype == "train": load_path = "/home/bhatti/dataset/VCHAMPS/death_train.csv"
elif datatype == "test": load_path = "/data/public/MLA/VCHAMPS-Test/death_test.csv"
death = pd.read_csv(load_path, index_col=0)

In [80]:
in_ad = pd.read_csv("inpatient_full_simple.csv")

In [81]:
for i in ed.columns:
    print(i, ed[ed[i].isna()].shape[0])

Internalpatientid 0
Age at ed visit 0
Ed visit start date 0
Discharge date ed 19
Died during ed visit 0
First listed diagnosis icd10 subcategory 0
Second listed diagnosis icd10 subcategory 0
State 0


### Check correlation between inpatient dataset and ed visits dataste

In [82]:
print("These patients are in inpatient dataset but not in ed_visits")
print(list(set(in_ad["Internalpatientid"].unique()) - set(ed["Internalpatientid"].unique())))

print("These patients are in ed dataset but not in inpatient")
print(list(set(ed["Internalpatientid"].unique()) - set(in_ad["Internalpatientid"].unique())))

These patients are in inpatient dataset but not in ed_visits
[1, 2, 3, 4, 5, 8, 10, 11, 13, 15, 16, 18, 19, 21, 23, 24, 27, 28, 30, 32, 35, 36, 37, 38, 39, 44, 47, 49, 50, 51, 52, 55, 56, 57, 61, 62, 70, 71, 72, 73, 76, 79, 82, 84, 87, 88, 89, 91, 92, 93, 94, 98, 99, 101, 103, 104, 111, 112, 113, 115, 117, 119, 120, 121, 122, 123, 124, 129, 130, 131, 132, 133, 135, 137, 139, 141, 143, 144, 151, 154, 155, 156, 161, 163, 165, 168, 171, 172, 175, 176, 177, 180, 181, 184, 185, 186, 188, 189, 191, 192, 194, 195, 197, 201, 204, 205, 206, 207, 209, 212, 218, 219, 221, 229, 231, 232, 238, 240, 241, 243, 245, 246, 249, 252, 257, 258, 260, 264, 265, 266, 268, 270, 272, 274, 275, 276, 277, 280, 284, 285, 287, 290, 292, 293, 297, 301, 302, 303, 304, 308, 311, 314, 315, 316, 321, 322, 328, 331, 332, 333, 335, 340, 341, 342, 344, 349, 352, 353, 355, 357, 359, 361, 363, 365, 366, 369, 370, 374, 375, 377, 380, 383, 385, 386, 388, 389, 391, 393, 396, 398, 400, 402, 403, 406, 408, 411, 412, 413, 415, 41

In [83]:
print("There are")
print(len(list(set(in_ad["Internalpatientid"].unique()) & set(ed["Internalpatientid"].unique()))))
print("patients that are both in inpatient and ed_visits dataset")

There are
0
patients that are both in inpatient and ed_visits dataset


### Preprocessing

In [84]:
ed

,Internalpatientid,Age at ed visit,Ed visit start date,Discharge date ed,Died during ed visit,First listed diagnosis icd10 subcategory,Second listed diagnosis icd10 subcategory,State
1,102421,69.520563,2017-11-06 18:22:30.0,2017-11-07 12:11:15.0,No,Systolic (congestive) heart failure,"Urinary tract infection, site not specified",New York
2,104374,44.119446,2020-09-23 15:46:58.0,2020-09-23 22:44:24.0,No,"Pneumonia, unspecified organism",Type 2 diabetes mellitus with ophthalmic compl...,Virginia
8,115748,63.721484,2020-05-02 04:51:35.0,2020-05-02 06:54:34.0,No,Other difficulties with micturition,"Urinary tract infection, site not specified",Florida
10,117891,74.097866,2023-05-10 18:28:53.0,2023-05-11 02:58:23.0,No,"Sepsis, unspecified organism","Urinary tract infection, site not specified",Arizona
18,119001,72.116844,2020-01-05 16:57:31.0,2020-01-05 21:58:47.0,No,Mechanical complication of urinary catheter,Other surgical procedures as the cause of abno...,California
...,...,...,...,...,...,...,...,...
701941,99825,56.253670,2009-04-14 17:11:34.0,2009-04-15 01:40:15.0,No,Not specified,Not specified,North Carolina
701943,99844,63.209756,2013-05-05 03:58:53.0,2013-05-05 07:02:06.0,No,Not specified,Not specified,Puerto Rico
701956,9996,84.150571,2013-02-12 00:48:28.0,2013-02-12 05:37:28.0,No,Not specified,Not specified,Pennsylvania
701957,99967,74.883312,2011-03-19 20:25:01.0,2011-03-19 23:50:56.0,No,Not specified,Not specified,Colorado


#### Age bins

In [85]:
ed['Age 20-40'] = [0] * len(ed)
ed['Age 40-60'] = [0] * len(ed)
ed['Age 60-80'] = [0] * len(ed)
ed['Age 80-100'] = [0] * len(ed)
ed['Age 100-120'] = [0] * len(ed)
def age_category(data):
    data.loc[(data['Age at ed visit'] > 20) & (data['Age at ed visit'] <= 40), 'Age 20-40'] = 1
    data.loc[(data['Age at ed visit'] > 40) & (data['Age at ed visit'] <= 60), 'Age 40-60'] = 1
    data.loc[(data['Age at ed visit'] > 60) & (data['Age at ed visit'] <= 80), 'Age 60-80'] = 1
    data.loc[(data['Age at ed visit'] > 80) & (data['Age at ed visit'] <= 100), 'Age 80-100'] = 1
    data.loc[(data['Age at ed visit'] > 100) & (data['Age at ed visit'] <= 120), 'Age 100-120'] = 1
    return data
ed = age_category(ed)
ed

,Internalpatientid,Age at ed visit,Ed visit start date,Discharge date ed,Died during ed visit,First listed diagnosis icd10 subcategory,Second listed diagnosis icd10 subcategory,State,Age 20-40,Age 40-60,Age 60-80,Age 80-100,Age 100-120
1,102421,69.520563,2017-11-06 18:22:30.0,2017-11-07 12:11:15.0,No,Systolic (congestive) heart failure,"Urinary tract infection, site not specified",New York,0,0,1,0,0
2,104374,44.119446,2020-09-23 15:46:58.0,2020-09-23 22:44:24.0,No,"Pneumonia, unspecified organism",Type 2 diabetes mellitus with ophthalmic compl...,Virginia,0,1,0,0,0
8,115748,63.721484,2020-05-02 04:51:35.0,2020-05-02 06:54:34.0,No,Other difficulties with micturition,"Urinary tract infection, site not specified",Florida,0,0,1,0,0
10,117891,74.097866,2023-05-10 18:28:53.0,2023-05-11 02:58:23.0,No,"Sepsis, unspecified organism","Urinary tract infection, site not specified",Arizona,0,0,1,0,0
18,119001,72.116844,2020-01-05 16:57:31.0,2020-01-05 21:58:47.0,No,Mechanical complication of urinary catheter,Other surgical procedures as the cause of abno...,California,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
701941,99825,56.253670,2009-04-14 17:11:34.0,2009-04-15 01:40:15.0,No,Not specified,Not specified,North Carolina,0,1,0,0,0
701943,99844,63.209756,2013-05-05 03:58:53.0,2013-05-05 07:02:06.0,No,Not specified,Not specified,Puerto Rico,0,0,1,0,0
701956,9996,84.150571,2013-02-12 00:48:28.0,2013-02-12 05:37:28.0,No,Not specified,Not specified,Pennsylvania,0,0,0,1,0
701957,99967,74.883312,2011-03-19 20:25:01.0,2011-03-19 23:50:56.0,No,Not specified,Not specified,Colorado,0,0,1,0,0


#### Died

In [86]:
ed["Died during ed visit"] = ed["Died during ed visit"].replace({"No":0, "Yes": 1})

#### Length of Visits

In [87]:
ed['Discharge date ed'] = pd.to_datetime(ed['Discharge date ed'])
ed['Ed visit start date'] = pd.to_datetime(ed['Ed visit start date'])
ed["stay_length"] = (ed['Discharge date ed'] - ed['Ed visit start date']).dt.days + round((ed['Discharge date ed'] - ed['Ed visit start date']).dt.seconds/3600/24,2)
ed

,Internalpatientid,Age at ed visit,Ed visit start date,Discharge date ed,Died during ed visit,First listed diagnosis icd10 subcategory,Second listed diagnosis icd10 subcategory,State,Age 20-40,Age 40-60,Age 60-80,Age 80-100,Age 100-120,stay_length
1,102421,69.520563,2017-11-06 18:22:30,2017-11-07 12:11:15,0,Systolic (congestive) heart failure,"Urinary tract infection, site not specified",New York,0,0,1,0,0,0.74
2,104374,44.119446,2020-09-23 15:46:58,2020-09-23 22:44:24,0,"Pneumonia, unspecified organism",Type 2 diabetes mellitus with ophthalmic compl...,Virginia,0,1,0,0,0,0.29
8,115748,63.721484,2020-05-02 04:51:35,2020-05-02 06:54:34,0,Other difficulties with micturition,"Urinary tract infection, site not specified",Florida,0,0,1,0,0,0.09
10,117891,74.097866,2023-05-10 18:28:53,2023-05-11 02:58:23,0,"Sepsis, unspecified organism","Urinary tract infection, site not specified",Arizona,0,0,1,0,0,0.35
18,119001,72.116844,2020-01-05 16:57:31,2020-01-05 21:58:47,0,Mechanical complication of urinary catheter,Other surgical procedures as the cause of abno...,California,0,0,1,0,0,0.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701941,99825,56.253670,2009-04-14 17:11:34,2009-04-15 01:40:15,0,Not specified,Not specified,North Carolina,0,1,0,0,0,0.35
701943,99844,63.209756,2013-05-05 03:58:53,2013-05-05 07:02:06,0,Not specified,Not specified,Puerto Rico,0,0,1,0,0,0.13
701956,9996,84.150571,2013-02-12 00:48:28,2013-02-12 05:37:28,0,Not specified,Not specified,Pennsylvania,0,0,0,1,0,0.20
701957,99967,74.883312,2011-03-19 20:25:01,2011-03-19 23:50:56,0,Not specified,Not specified,Colorado,0,0,1,0,0,0.14


In [88]:
# nan_admission = ed[ed["stay_length"].isna()]
# ed = ed[~ed["stay_length"].isna()]
# nan_admission["stay_length"] = ed["stay_length"].mean()
# ed = pd.concat([ed, nan_admission])
# ed

### Cardiovascular

In [89]:
cardiovascular = ["Heart failure, unspecified", "Other heart failure",
                  "Cardiogenic shock", "Hypertensive heart disease with heart failure",
                  "Hypertensive heart and kidney disease with heart failure", "Unstable angina", "Other forms of chronic ischemic heart disease",
                  "Atherosclerotic heart disease of native coronary artery", "Atrial fibrillation", "Atrial flutter",
                  "Supraventricular tachycardia", "Ventricular tachycardia"]

In [90]:
ed["cd_diagnosis"] = [0] * len(ed)
def cd_diagnosis(data):
    data.loc[(ed["Second listed diagnosis icd10 subcategory"].str.contains('|'.join(cardiovascular))
      | ed["Second listed diagnosis icd10 subcategory"].str.contains("Systolic (congestive) heart failure", regex=False)
      | ed["Second listed diagnosis icd10 subcategory"].str.contains("Diastolic (congestive) heart failure", regex=False)
      | ed["Second listed diagnosis icd10 subcategory"].str.contains("Combined systolic (congestive) and diastolic (congestive) heart failure", regex=False)
      | ed["Second listed diagnosis icd10 subcategory"].str.contains("ST elevation (STEMI) myocardial infarction", regex=False)
      | ed["Second listed diagnosis icd10 subcategory"].str.contains("Non-ST elevation (NSTEMI) myocardial infarction", regex=False)
      | ed["First listed diagnosis icd10 subcategory"].str.contains('|'.join(cardiovascular))
      | ed["First listed diagnosis icd10 subcategory"].str.contains("Systolic (congestive) heart failure", regex=False)
      | ed["First listed diagnosis icd10 subcategory"].str.contains("Diastolic (congestive) heart failure", regex=False)
      | ed["First listed diagnosis icd10 subcategory"].str.contains("Combined systolic (congestive) and diastolic (congestive) heart failure", regex=False)
      | ed["First listed diagnosis icd10 subcategory"].str.contains("ST elevation (STEMI) myocardial infarction", regex=False)
      | ed["First listed diagnosis icd10 subcategory"].str.contains("Non-ST elevation (NSTEMI) myocardial infarction", regex=False)), "cd_diagnosis"] = 1
    return data
ed = cd_diagnosis(ed)

In [91]:
ed.columns

Index(['Internalpatientid', 'Age at ed visit', 'Ed visit start date',
       'Discharge date ed', 'Died during ed visit',
       'First listed diagnosis icd10 subcategory',
       'Second listed diagnosis icd10 subcategory', 'State', 'Age 20-40',
       'Age 40-60', 'Age 60-80', 'Age 80-100', 'Age 100-120', 'stay_length',
       'cd_diagnosis'],
      dtype='object')

### Use RF to impute stay_length

In [92]:
ed["State"] = ed["State"].astype("category")
ed["State_encoding"] = ed["State"].cat.codes

In [93]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

no_na = ed[ed['stay_length'].notna()]
only_na = ed[ed['stay_length'].isna()]

# use upsampled data to train the model
X = no_na[['Age at ed visit', 'cd_diagnosis', "State_encoding"]].values
Y = no_na['stay_length'].values


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

forest = RandomForestRegressor(n_estimators=100)

forest.fit(X_train, Y_train)

y_pred_train = forest.predict(X_train)
y_pred = forest.predict(X_test)

# y_pred_proba = forest.predict_proba(X_test)

accuracy_train = mean_squared_error(Y_train, y_pred_train)
accuracy_test = mean_squared_error(Y_test, y_pred)

print("ERROR: TRAIN=%.4f TEST=%.4f" % (accuracy_train,accuracy_test))
# print("LOG LOSS: "+str(log_loss(Y_test, y_pred_proba)))

ERROR: TRAIN=20.2736 TEST=707.7789


In [94]:
only_na = ed[ed['stay_length'].isna()]
only_na = only_na[['Age at ed visit', 'cd_diagnosis', "State_encoding"]].values
prediction = forest.predict(only_na)
only_na = ed[ed['stay_length'].isna()].drop('stay_length', axis=1)
only_na['stay_length'] = prediction

ed = [only_na, no_na]
ed = pd.concat(ed)

In [95]:
ed = ed[~ed["Discharge date ed"].isna()]
ed = ed.sort_values(by=["Internalpatientid", "Ed visit start date"])
ed

,Internalpatientid,Age at ed visit,Ed visit start date,Discharge date ed,Died during ed visit,First listed diagnosis icd10 subcategory,Second listed diagnosis icd10 subcategory,State,Age 20-40,Age 40-60,Age 60-80,Age 80-100,Age 100-120,cd_diagnosis,State_encoding,stay_length
678411,7,64.228174,2010-09-29 09:32:24,2010-09-29 11:20:04,0,Not specified,Not specified,Nebraska,0,0,1,0,0,0,23,0.07
688376,7,66.771904,2013-04-16 03:08:28,2013-04-16 09:38:17,0,Not specified,Not specified,Nebraska,0,0,1,0,0,0,23,0.27
538631,7,67.705070,2014-03-23 04:52:21,2014-03-23 10:27:41,0,Other abdominal pain,"Nausea with vomiting, unspecified",Nebraska,0,0,1,0,0,0,23,0.23
639220,7,69.524108,2016-01-17 01:28:18,2016-01-17 06:19:38,0,Dehydration,Weakness,Nebraska,0,0,1,0,0,0,23,0.20
411580,7,70.916759,2017-06-09 01:48:21,2017-06-09 01:50:36,0,Not specified,Not specified,New Mexico,0,0,1,0,0,0,27,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214861,169037,83.295943,2013-05-13 16:00:54,2013-05-13 18:28:58,0,"Retention of urine, unspecified",Not specified,Illinois,0,0,0,1,0,0,11,0.10
552183,169037,83.346190,2013-06-01 00:46:35,2013-06-01 04:03:30,0,Other specified disorders of muscle,Unspecified atrial fibrillation and atrial flu...,Illinois,0,0,0,1,0,0,11,0.14
640252,169037,85.026314,2015-02-05 02:49:44,2015-02-05 05:48:10,0,Unspecified fracture of femur,Other fall from one level to another,Illinois,0,0,0,1,0,0,11,0.12
234731,169065,53.148782,2011-04-09 21:07:12,2011-04-11 05:41:53,0,Not specified,Not specified,Arizona,0,1,0,0,0,0,1,1.36


### Final dataset

In [96]:
def final_set_before(ids, group):
    readmission = 0
    num_admissions = group["Age at ed visit"].nunique()
    if num_admissions > 1: readmission=1

    age_mean = group["Age at ed visit"].mean()
    age_std = group["Age at ed visit"].std()
    if group["Age at ed visit"].nunique() == 1: age_std = 0

    min_age = group["Age at ed visit"].min()
    max_age = group["Age at ed visit"].max()
    freq = num_admissions/(math.floor(max_age - min_age) + 1)

    min_stay = group["stay_length"].min()
    max_stay = group["stay_length"].min()
    stay_mean = group["stay_length"].mean()
    stay_std = group["stay_length"].std()
    if group["stay_length"].nunique() == 1: stay_std = 0

    num_cvd_readmission = max(0, group['cd_diagnosis'].sum() - 1)

    cvd = 0
    if group["cd_diagnosis"].sum() > 0: cvd = 1
    
    Died = 0
    if group["Died during ed visit"].sum() > 0: Died = 1

    df = pd.DataFrame(data={'Internalpatientid': [ids], 'num_stays': [num_admissions], 'stay_length': group["stay_length"].sum(),
                            "num_cvd_readmission": [num_cvd_readmission], "Readmission":[readmission], "Died": [Died], "CVD": [cvd],
                            'Age 20-40': group["Age 20-40"].sum(), 'Age 40-60': group["Age 40-60"].sum(), 'Age 60-80':group["Age 60-80"].sum(), 
                            'Age 80-100':group["Age 80-100"].sum(), 'Age 100-120':group["Age 100-120"].sum(), "age_mean": [age_mean], "age_std": [age_std], 
                            "age_min": [min_age], "age_max": [max_age], "stay_min": [min_stay], "stay_max": [max_stay], "stay_mean": [stay_mean],
                            "stay_std": [stay_std], "freq": [round(freq,2)], "Discharge date": group.iloc[-1,:]['Discharge date ed']
                            })
    
    df = df.reset_index(drop=True)
    return df

In [97]:
tidy_dataset = []
for ids, group in tqdm(ed.groupby("Internalpatientid")):
    group = group.sort_values(by = ['Discharge date ed'],ascending=True).reset_index(drop = True)
    df = final_set_before(ids, group)
    tidy_dataset.append(df)
    
tidy_dataset = pd.concat(tidy_dataset)

  0%|          | 0/15368 [00:00<?, ?it/s]

### Add death indicator

In [98]:
tidy_dataset = tidy_dataset.merge(death, how="left",on="Internalpatientid")

In [99]:
tidy_dataset['Discharge date'] = pd.to_datetime(tidy_dataset['Discharge date'])
tidy_dataset['Death date'] = pd.to_datetime(tidy_dataset['Death date'])
tidy_dataset["away_from_death"] = (tidy_dataset['Death date'] - tidy_dataset['Discharge date']).dt.days

In [100]:
tidy_dataset['died_within_900days'] = [0] * len(tidy_dataset)

def age_category(data):
    data.loc[(data['away_from_death'] < 900), 'died_within_900days'] = 1
    data.loc[(data['away_from_death'] >= 900), 'died_within_900days'] = 0
    data.loc[(data['away_from_death'].isna()), 'died_within_900days'] = 0
    return data
tidy_dataset = age_category(tidy_dataset)
tidy_dataset

,Internalpatientid,num_stays,stay_length,num_cvd_readmission,Readmission,Died,CVD,Age 20-40,Age 40-60,Age 60-80,...,stay_min,stay_max,stay_mean,stay_std,freq,Discharge date,Age at death,Death date,away_from_death,died_within_900days
0,7,5,0.77,0,1,0,0,0,0,5,...,0.00,0.00,0.154000,0.114149,0.71,2017-06-09 01:50:36,NaN,NaT,NaN,0
1,22,2,0.56,0,1,0,0,0,0,2,...,0.17,0.17,0.280000,0.155563,2.00,2014-05-08 01:49:17,61.533334,2014-06-13 10:24:14,36.0,1
2,25,12,1.91,0,1,0,0,0,0,12,...,0.10,0.10,0.159167,0.056481,1.50,2023-05-22 12:05:22,NaN,NaT,NaN,0
3,41,4,0.54,0,1,0,0,0,0,4,...,0.06,0.06,0.135000,0.061373,1.33,2017-03-07 23:18:07,64.013190,2017-07-26 19:42:15,140.0,1
4,42,5,0.54,0,1,0,0,0,1,4,...,0.05,0.05,0.108000,0.056303,0.56,2018-12-25 05:44:25,67.910649,2019-01-22 08:12:38,28.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15363,168956,2,0.29,0,1,0,1,0,0,2,...,0.02,0.02,0.145000,0.176777,2.00,2020-06-12 18:24:36,NaN,NaT,NaN,0
15364,168995,22,3.22,0,1,0,0,0,0,22,...,0.03,0.03,0.146364,0.108257,2.20,2021-08-15 05:39:54,75.662580,2021-08-21 16:08:52,6.0,1
15365,169011,2,3101.98,0,1,1,0,0,0,2,...,0.21,0.21,1550.990000,2193.134108,1.00,2022-06-04 10:28:49,75.187690,2018-11-28 02:24:50,-1285.0,1
15366,169037,5,0.56,0,1,0,0,0,0,1,...,0.04,0.04,0.112000,0.046043,0.83,2015-02-05 05:48:10,87.743394,2017-10-25 05:03:14,992.0,0


In [101]:
if datatype == "train": tidy_dataset.to_csv("ed_visits_full.csv")
elif datatype == "test": tidy_dataset.to_csv("ed_visits_test.csv")
else: tidy_dataset.to_csv("ed_visits_quality.csv")